In [270]:
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

In [215]:
page_url = "https://kworb.net/spotify/"

uClient = uReq(page_url) # downloads the html page from page_url

page_soup = soup(uClient.read(), "html.parser") # parses html into a readable alphabet soup

country_soup = page_soup.find("table") # narrows html down to the country

country_soup_list = country_soup.find_all("tr") # get each row of the country soup

country_dict_links = {}

for entry in country_soup_list:
    
    country_name = entry.contents[0].string # country name
    country_links = []
    for link in entry.contents[2].find_all("a"):
        
        country_links.append(link.get('href'))
    country_dict_links[country_name] = country_links

In [199]:
country_abbv = {}
for country in country_dict_links.keys():
    daily_link = country_dict_links[country][0]
    match = re.search(r'country/(\w+)_daily.html', daily_link)
    abbv = match.group(1)
    country_abbv[country] = abbv

help

In [220]:
page_url = "https://kworb.net/spotify/country/global_daily.html"

uClient = uReq(page_url) # downloads the html page from page_url

page_soup = soup(uClient.read(), "html.parser") # parses html into a readable alphabet soup

spotify_entry_soup = page_soup.find("table") # narrows html down to the country

In [345]:
col_names = [col_soup.string for col_soup in spotify_entry_soup.thead.contents[0].find_all("th")]
col_names.remove('Artist and Title')
col_names.remove("(x?)")

In [346]:
col_names

['Pos', 'P+', 'Days', 'Pk', 'Streams', 'Streams+', '7Day', '7Day+', 'Total']

In [412]:
headings = [th.get_text() for th in spotify_entry_soup.find("tr").find_all("th")]

datasets = []
for row in spotify_entry_soup.find_all("tr")[1:]:
    dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
    datasets.append(dataset)

In [413]:
dataset_tuples = [list(zip(dataset)) for dataset in datasets]

In [414]:
dataset_tuples

[[(('Pos', '1'),),
  (('P+', '+1'),),
  (('Artist and Title', 'Tate McRae - greedy'),),
  (('Days', '147'),),
  (('Pk', '1'),),
  (('(x?)', '(x34)'),),
  (('Streams', '5,102,204'),),
  (('Streams+', '-114,147'),),
  (('7Day', '35,438,103'),),
  (('7Day+', '-65,405'),),
  (('Total', '738,388,826'),)],
 [(('Pos', '2'),),
  (('P+', '+1'),),
  (('Artist and Title', 'Xavi - La Diabla'),),
  (('Days', '63'),),
  (('Pk', '1'),),
  (('(x?)', '(x20)'),),
  (('Streams', '4,968,264'),),
  (('Streams+', '-24,017'),),
  (('7Day', '36,184,432'),),
  (('7Day+', '-426,914'),),
  (('Total', '280,997,027'),)],
 [(('Pos', '3'),),
  (('P+', '-2'),),
  (('Artist and Title', 'Benson Boone - Beautiful Things'),),
  (('Days', '21'),),
  (('Pk', '1'),),
  (('(x?)', '(x3)'),),
  (('Streams', '4,814,921'),),
  (('Streams+', '-462,102'),),
  (('7Day', '34,070,246'),),
  (('7Day+', '+426,294'),),
  (('Total', '76,058,079'),)],
 [(('Pos', '4'),),
  (('P+', '='),),
  (('Artist and Title',
    'The Weeknd - One Of Th

In [420]:
# Convert the data into a dictionary to handle repeated attributes
dict_data = {}
for sublist in dataset_tuples:
    for tup in sublist:
        attribute, value = tup[0]
        if attribute in dict_data:
            dict_data[attribute].append(value)
        else:
            dict_data[attribute] = [value]

# Create a DataFrame from the dictionary
df = pd.DataFrame(dict_data)
df = df.set_index("Pos")

In [441]:
df[['Artist', 'Title']] = df["Artist and Title"].str.split("-", 1, expand=True)

In [444]:
df_obj = df.select_dtypes('object')
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [447]:
df.reset_index()

,Pos,P+,Artist and Title,Days,Pk,(x?),Streams,Streams+,7Day,7Day+,Total,Artist,Title
0,1,+1,Tate McRae - greedy,147,1,(x34),"5,102,204","-114,147","35,438,103","-65,405","738,388,826",Tate McRae,greedy
1,2,+1,Xavi - La Diabla,63,1,(x20),"4,968,264","-24,017","36,184,432","-426,914","280,997,027",Xavi,La Diabla
2,3,-2,Benson Boone - Beautiful Things,21,1,(x3),"4,814,921","-462,102","34,070,246","+426,294","76,058,079",Benson Boone,Beautiful Things
3,4,=,"The Weeknd - One Of The Girls (w/ JENNIE, Lily...",124,2,(x2),"4,786,621","-147,152","33,826,703","-287,326","370,855,105",The Weeknd,"One Of The Girls (w/ JENNIE, Lily-Rose Depp)"
4,5,=,Taylor Swift - Cruel Summer,368,2,(x17),"4,517,091","+47,529","30,425,874","+219,997","1,267,250,653",Taylor Swift,Cruel Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,-18,Milo j - M.A.I,11,141,,"1,240,359","-49,223","9,037,556","-139,481","14,140,746",Milo j,M.A.I
196,197,+3,Drake - First Person Shooter (w/ J. Cole),120,2,(x1),"1,239,119","-3,496","6,134,743","-22,305","211,830,969",Drake,First Person Shooter (w/ J. Cole)
197,198,+1,Bruno Mars - That's What I Like,695,5,,"1,238,069","-4,949","5,992,025","+1,238,069","848,450,877",Bruno Mars,That's What I Like
198,199,-16,Taylor Swift - Karma,283,8,,"1,236,518","-37,417","5,163,086","+1,236,518","487,247,233",Taylor Swift,Karma


In [421]:
pd.DataFrame(test[0], position=test[0][0])

TypeError: __init__() got an unexpected keyword argument 'position'

In [316]:
top_charts = {}
position = 1
for artist_title in spotify_entry_soup.tbody.find_all("a"):
    collabs = []
    switch = None
    if "/artist/" in artist_title.get('href'):
        if switch != "/artist/":
            artist = artist_title.string
            switch = "/artist/"
        else:
            collabs.append(artist_title.string)
    if "/track/" in artist_title.get('href'):
        title = artist_title.string
        switch = "/track/"
    else:
        collabs.append(artist_title.string)
    
    if len(collabs) == 0:
        top_charts[position] = {"Artist": artist, "Title": title}
        position += 1

In [317]:
top_charts

{1: {'Artist': 'Tate McRae', 'Title': 'greedy'},
 2: {'Artist': 'Xavi', 'Title': 'La Diabla'},
 3: {'Artist': 'Benson Boone', 'Title': 'Beautiful Things'},
 4: {'Artist': 'The Weeknd', 'Title': 'One Of The Girls'},
 5: {'Artist': 'Taylor Swift', 'Title': 'Cruel Summer'},
 6: {'Artist': 'Mitski', 'Title': 'My Love Mine All Mine'},
 7: {'Artist': '21 Savage', 'Title': 'redrum'},
 8: {'Artist': 'Noah Kahan', 'Title': 'Stick Season'},
 9: {'Artist': 'Feid', 'Title': 'LUNA'},
 10: {'Artist': 'Jack Harlow', 'Title': 'Lovin On Me'},
 11: {'Artist': 'Teddy Swims', 'Title': 'Lose Control'},
 12: {'Artist': 'Ariana Grande', 'Title': 'yes, and?'},
 13: {'Artist': 'The Weeknd', 'Title': 'Popular'},
 14: {'Artist': 'Miley Cyrus', 'Title': 'Flowers'},
 15: {'Artist': 'YG Marley', 'Title': 'Praise Jah In The Moonlight'},
 16: {'Artist': 'Xavi', 'Title': 'La Víctima'},
 17: {'Artist': 'Kali Uchis', 'Title': 'Igual Que Un Ángel'},
 18: {'Artist': 'Natasha Bedingfield', 'Title': 'Unwritten'},
 19: {'Art

maybe add today streams and total streams

In [366]:
top_200_global_df = pd.DataFrame.from_dict(top_charts, orient='index')
top_200_global_df.to_csv("top_200_global_df.csv", index=False)